In [69]:
import numpy as np
from collections import defaultdict

In [95]:
class LINQ_list:
    def __init__(self, data):
        self._data = iter(data)
    
    def ToList(self):
        return list(self._data)
    
    def Select(self, operation):
        return LINQ_list(operation(elem) for elem in self._data)
            
    def Where(self, predicate):                
        return LINQ_list(elem for elem in self._data if predicate(elem))
    
    def Flatten(self):
        return LINQ_list(elem for elems in self._data for elem in elems)
    
    def Take(self, k):
        def ret():
            for i in range(k):
                yield next(self._data)
        return LINQ_list(ret())
    
    
    class LazyContainerIterator:
        def __init__(self, fun_generator):
            self._fun_generator = fun_generator
            self._iterator = None
            
        def __next__(self):
            if self._iterator is None:
                self._iterator = iter(self._fun_generator())
            return next(self._iterator)
                
        def __iter__(self):
            return self
    
    def GroupBy(self, key_calculator):
        def fun():
            groups_dict = defaultdict(list)
            for x in self._data:
                groups_dict[key_calculator(x)].append(x)
            return groups_dict.items()
        return LINQ_list(self.LazyContainerIterator(fun))
            
    def OrderBy(self, sorter):
        return LINQ_list(sorted(self._data, key=sorter))

In [93]:
# def f():
#     yield 1
#     yield 2
    
# a = iter(f())
# print(next(a))
# print(next(a))
# print(next(a))

In [94]:
l = LINQ_list(range(3))
print(l.GroupBy(lambda x : x % 2).ToList())

[(0, [0, 2]), (1, [1])]


In [63]:
l = LINQ_list(range(3))
l.Select(lambda x : x * x).Where(lambda x: x > 0).ToList()
#next(gen)

[1, 4]

In [96]:
l1 = LINQ_list([[1, 2, 3], [4, 5]])
l1.Flatten().Take(4).OrderBy(lambda x : -x).ToList()

[4, 3, 2, 1]

In [97]:
def fibonacci_generator():
    a, b = 1, 1
    while True:
        yield a;
        a, b = b, a + b

In [100]:
fib = LINQ_list(fibonacci_generator())
fib.Where(lambda x : x % 3 == 0).Select(lambda x: x * x if x % 2 == 0 else x).Take(5).ToList()

[3, 21, 20736, 987, 6765]

In [108]:
string = "lalala lala wwergw lala hello hello\n hello hello lala"
strings = string.split("\n")

word_count = LINQ_list(strings)
word_count.Select(str.split).Flatten().GroupBy(lambda x: x).Select(lambda pair: (pair[0], len(pair[1])))\
.OrderBy(lambda pair : -pair[1]).ToList()

[('hello', 4), ('lala', 3), ('lalala', 1), ('wwergw', 1)]

In [110]:
with open("text.txt", 'r') as f:
    file_strings = LINQ_list(f.readlines())
    print(file_strings.Select(str.split).Flatten().GroupBy(lambda x: x).Select(lambda pair: (pair[0], len(pair[1])))\
        .OrderBy(lambda pair : -pair[1]).ToList())

[('в', 13), ('Российской', 12), ('по', 11), ('Федерации', 8), ('и', 7), ('года', 7), ('с', 6), ('7', 4), ('2012', 4), ('1999', 4), ('был', 4), ('—', 3), ('мая', 3), ('Федерации.', 3), ('безопасности', 3), ('С', 3), ('работал', 3), ('президента', 3), ('государственный', 2), ('2000', 2), ('2008', 2), ('В', 2), ('годах', 2), ('Правительства', 2), ('директора', 2), ('секретаря', 2), ('Совета', 2), ('Ленинградского', 2), ('должности', 2), ('августа', 2), ('на', 2), ('назначен', 2), ('государства', 2), ('России', 2), ('(1997).', 2), ('самбо', 2), ('дзюдо', 2), ('Влади́мир', 1), ('Влади́мирович', 1), ('Пу́тин', 1), ('(род.', 1), ('октября', 1), ('1952,', 1), ('Ленинград,', 1), ('РСФСР,', 1), ('СССР[5][6])', 1), ('российский', 1), ('политический', 1), ('деятель,', 1), ('действующий', 1), ('Президент', 1), ('Федерации,', 1), ('Верховный', 1), ('Главнокомандующий', 1), ('Вооружёнными', 1), ('силами', 1), ('года[7][8].', 1), ('1999—2000', 1), ('годы', 1), ('Председатель', 1), ('Занимал', 1), ('по